성능 최적화를 위해 텐서 내의 모든 객체는 같은 타입의 숫자여야 하고, 파이토치는 실행 중에 이런 숫자 타입을 계속 추첮하고 있어야 한다.

텐서의 기본 데이터 타입은 32비트 부동소수점이다.

In [1]:
import torch

double_points = torch.ones(10, 2, dtype=torch.double)
double_points

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]], dtype=torch.float64)

In [2]:
short_points = torch.tensor([[1,2], [3,4]], dtype=torch.short)
short_points

tensor([[1, 2],
        [3, 4]], dtype=torch.int16)

In [3]:
double_points = torch.zeros(10, 2).double()
double_points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]], dtype=torch.float64)

In [4]:
double_points = torch.zeros(10, 2).to(torch.double)
double_points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]], dtype=torch.float64)

In [5]:
points_64 = torch.randn(5, dtype=torch.double)
print(points_64)
points_short = points_64.to(torch.short)
print(points_short)
points_64 * points_short # 파이토치 1.3 부터

tensor([-1.4511,  0.8236, -1.2383, -1.2843,  0.2402], dtype=torch.float64)
tensor([-1,  0, -1, -1,  0], dtype=torch.int16)


tensor([1.4511, 0.0000, 1.2383, 1.2843, 0.0000], dtype=torch.float64)

```
torch.transpose(input, dim0, dim1) → Tensor

In [6]:
a = torch.ones(3,2)
a

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])

In [7]:
a_t = torch.transpose(a, 0, 1)
a_t

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [8]:
a_t = a_t.transpose(1, 0)
a_t

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])

탠서 내부 값은 실제로는 torch.Storage인스턴스로 관리한다.   
텐서는 storage에 담긴 데이터를 보여주는 역할

In [9]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage()

/tmp/ipykernel_62664/2923139482.py:2: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  points.storage()


 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [10]:
points_storage = points.storage()
print(points_storage[0])
print(points.storage()[1])

4.0
1.0


텐서는 참조 하고 있는 것이기 때문에 storage 값을 바꾸면 텐서도 바뀜

In [11]:
points_storage[0] = 2.0
points

tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])

zero_() 처럼 _로 끝나는 함수들은 변수에 저장된 값들을 바꾸는 역할을 한다.   
_로 끝나지 않는 모든 메소드들은 원래 텐서는 그대로 두고 새로운 텐서를 만들어 넘겨준다.

In [12]:
a = torch.ones(3,2)
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

size(shape) : 텐서의 각 차원별로 들어가는 요소의 수를 표시한 튜플   
offset : 텐서의 첫 번째 요소를 가리키는 색인 값   
stride : 각 차원에서 다음 요소를 가리키고 싶을 떄 실제 저장 공간상에서 몇 개의 요소를 건너 뛰어야 하는지를 알려주는 숫자
```
point[i][j] = sotrage_offset + stride[0]*i + stride[1]*j

In [13]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_points = points[1]
second_points.storage_offset()

2

In [14]:
print(second_points.size())
print(second_points)

torch.Size([2])
tensor([5., 3.])


In [15]:
print(points.stride())
print(second_points.stride())

(2, 1)
(1,)


In [16]:
second_points[0] = 10.0
points

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

In [17]:
# 복사를 하면 서브텐서를 새 텐서로 복사해서 만듦
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_points = points[1].clone()
second_points[0] = 10.0
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

### torch.Tensor.data_ptr
```
Tensor.data_ptr() → int
```
Returns the address of the first element of self tensor.

[id 값에대한 stack overflow](https://stackoverflow.com/questions/67289617/pytorch-tensor-storages-have-the-same-id-when-calling-the-storage-method)

Python의 id()는 객체의 정체성을 반환한다. 이 id는 객체의 수명 동안 고유하고 일정하다.  
``` print(id(points.storage()) == id(second_points.storage())) ```   
여기서 points.storage()와 second_points.storage()는 호출 후 바로 사용이 끝나고, Python의 **가비지 콜렉터(Garbage Collector)**에 의해 삭제될 수 있다. 만약 points.storage()가 삭제된 후에 second_points.storage()가 생성된다면, 두 객체는 동일한 메모리 주소를 사용할 가능성이 있으므로 id가 같은 값을 가질 수 있다.   

하지만 가비지 콜렉터의 동작 타이밍이나 객체 생성 순서에 따라 다른 값을 가져서 False가 반환될 수도 있다.      

``` print(id(points.storage()) is id(second_points.storage())) ```   
다음 코드는 print가 끝나면 가비지 콜렉터가 수집하기 때문에 False가 나오게 된다.   

만약 둘이 같은 데이터인지 확인하려면, .data_ptr()을 사용해서 둘이 같은 요소를 공유하는지 확인하는 것이 나은 선택이 될 수 있다.

[is vs == 에 관한 stack overflow](https://stackoverflow.com/questions/132988/is-there-a-difference-between-and-is)   
다음 글에 따르면 is는 객체가 같은지를 확인하고, ==는 변수가 참조하는 객체가 같은지를 확인한다.

In [18]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_points = points[1]
# 둘다 같은 storage를 시작 요소의 주소값은 당연히 같다.
print(points.storage().data_ptr() == second_points.storage().data_ptr())
# ????
print(id(points.storage()) == id(second_points.storage()))
# 둘다 같은 storage이므로 값도 같다.
print(points.storage()[2] == second_points.storage()[2])

True
False
True


In [19]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_t = points.t()

a,b=id(points.storage()),id(points_t.storage())
print(a)
print(b)

128132029013872
128132029013680


In [20]:
points.storage().data_ptr()==points_t.storage().data_ptr()

True

In [21]:
points.is_contiguous()

True

In [22]:
points_t.is_contiguous()

False

In [23]:
# contiguous()함수를 사용하면 인접한 tensor를 만들 수 있다.
points_t_cont = points_t.contiguous()
points_t_cont

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [24]:
points_t_cont.is_contiguous()

True

In [25]:
print(points_t.stride())
print(points_t_cont.stride())

(1, 2)
(3, 1)


### GPU를 사용하기

In [26]:
# 모든 요소에 특정 상수를 곱하는 등의 텐서 연산은 GPU에서 수행된다.
points_gpu = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]], device='cuda')

In [27]:
points_gpu

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]], device='cuda:0')

In [28]:
point_gpu2 = 2 * points.to(device='cuda')
point_gpu2

tensor([[ 8.,  2.],
        [10.,  6.],
        [ 4.,  2.]], device='cuda:0')

In [29]:
points_gpu = points
print(point_gpu2)
points_gpu = points.cuda()
print(point_gpu2)
points_cpu = points_gpu.cpu()
print(points_cpu)

tensor([[ 8.,  2.],
        [10.,  6.],
        [ 4.,  2.]], device='cuda:0')
tensor([[ 8.,  2.],
        [10.,  6.],
        [ 4.,  2.]], device='cuda:0')
tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])
